In [24]:
import boto3
import pandas as pd

table_name = 'beacons'
client = boto3.client('dynamodb')


In [26]:

response = client.query(
    TableName=table_name,
     KeyConditionExpression='#src = :tracker_1 AND timestamp_ms BETWEEN :start AND :end', 
    ExpressionAttributeNames={
        '#src': 'source'  # Assuming 'source' is the name of your primary partition key
    },
    ExpressionAttributeValues={
        ':tracker_1': {'S': 'traker_1'} ,
        ':start': {'S': str(start_timestamp)},
        ':end': {'S': str(end_timestamp)} 
    }
)

In [83]:
start_timestamp = 1724251853000
end_timestamp   = 1724251853000 + 120000

items = response['Items']
beacons = []
for item in items:
    beacons.append([item['timestamp_ms']['S'], item['source']['S'], item['rssi']['N'], item['mac']['S']]) 

dtype = {'timestamp_ms': 'int64', 'source': 'str', 'rssi': 'int64', 'mac': 'str'}
df = pd.DataFrame(beacons, columns=['timestamp_ms', 'source', 'rssi', 'mac']).astype(dtype)

print(f'we have {len(df)} samples from', len(df['mac'].unique()), 'Beacons') 

we have 3 samples from 1 Beacons


In [90]:
treshold = 70 

for fr in range(len(df[['timestamp_ms' , 'rssi']])):
    print(df[['timestamp_ms' , 'rssi']])

 #   if 
#print(df[['timestamp_ms' , 'rssi']])

    timestamp_ms  rssi
0  1724251894209   -79
1  1724251894283   -82
2  1724251894407   -84
    timestamp_ms  rssi
0  1724251894209   -79
1  1724251894283   -82
2  1724251894407   -84
    timestamp_ms  rssi
0  1724251894209   -79
1  1724251894283   -82
2  1724251894407   -84


61.7037037037037

In [38]:
# print mac addresses for beacons inside the room and beacons outside the room
df_place = df.groupby(['mac']).max().sort_values(by='rssi', ascending=False)['rssi'] > -75
for mac in df_place.index:
    if df_place[mac]:
        print(f"beacon: {mac} is inside the room")
    else:
        print(f"beacon: {mac} is outside the room")

beacon: ea:10:2f:0e:f1:c5 is outside the room


In [80]:
df.groupby(['mac'])['rssi'].sort_values(ascending=False)

SyntaxError: invalid syntax (4139870026.py, line 1)

In [47]:
# bench dome experience
print(f'we have {len(df)} samples')

    timestamp_ms    source  rssi                mac
0  1724251894209  traker_1   -79  ea:10:2f:0e:f1:c5
1  1724251894283  traker_1   -82  ea:10:2f:0e:f1:c5
2  1724251894407  traker_1   -84  ea:10:2f:0e:f1:c5
we have 3 samples


In [31]:
df

,timestamp_ms,source,rssi,mac
0,1724251894209,traker_1,-79,ea:10:2f:0e:f1:c5
1,1724251894283,traker_1,-82,ea:10:2f:0e:f1:c5
2,1724251894407,traker_1,-84,ea:10:2f:0e:f1:c5


In [ ]:
# commit 1